# NLP - Bot basado en reglas con Tensorflow
Este ejemplo consiste en armar BOT simple basado en una red neuronal con Tensorflow

**Author:** Hernán Contigiani<br>
[Github](https://github.com/hernancontigiani/)<br>
[Linkedin](https://www.linkedin.com/in/hern%C3%A1n-contigiani-41260679/?locale=en_US)

In [ ]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

import pickle

In [ ]:
import os
import gdown
if os.access('lematizacion-es.pickle', os.F_OK) is False:
    if os.access('lematizacion-es.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=16leuM9PuFXAkmw34XeQy-84h8WGAYxJw&export=download'
        output = 'lematizacion-es.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q lematizacion-es.zip
else:
    print("El archivo ya se encuentra descargado")

In [ ]:
with open("lematizacion-es.pickle",'rb') as fi:
    lemma_lookupTable = pickle.load(fi)

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
# Dataset en formato JSON que representa las posibles preguntas (patterns)
# y las posibles respuestas por categoría (tag)
data = {"intents": [
             {"tag": "bienvenida",
              "patterns": ["Hola", "¿Cómo estás?", "¿Qué tal?"],
              "responses": ["Hola!", "Hola, ¿Cómo estás?"],
             },
             {"tag": "nombre",
              "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?"],
              "responses": ["Mi nombre es MarvelBOT", "Yo soy MarvelBOT"]
             },
            {"tag": "contacto",
              "patterns": ["contacto", "número de contacto", "número de teléfono", "número de whatsapp", "whatsapp"],
              "responses": ["Podes contactarnos al siguiente número +54-9-11-2154-4777", "Contactonos al whatsapp número +54-9-11-2154-4777"]
             },
            {"tag": "envios",
              "patterns": ["¿Realizan envios?", "¿Cómo me llega el paquete?"],
              "responses": ["Los envios se realizan por correo, lo enviaremos a la dirección que registraste en la página"]
             },
            {"tag": "precios",
              "patterns": ["precio", "Me podrás pasar los precios", "¿Cuánto vale?", "¿Cuánto sale?"],
              "responses": ["En el catálogo podrás encontrar los precios de todos nuestros productos en stock"]
             },
            {"tag": "pagos",
              "patterns": ["medios de pago", "tarjeta de crédito", "tarjetas", "cuotas"],
              "responses": ["Contactanos al whatsapp número +54-9-11-2154-4777 para conocer los beneficios y formas de pago vigentes"]
             },
            {"tag": "stock",
              "patterns": ["Esto está disponible", "¿Tenes stock?", "¿Hay stock?"],
              "responses": ["Los productos publicados están en stock"]
             },
            {"tag": "agradecimientos",
              "patterns": [ "Muchas gracias", "Gracias"],
              "responses": ["Por nada!, cualquier otra consulta podes escribirnos"]
             },
             {"tag": "despedida",
              "patterns": [ "Chau", "Hasta luego!"],
              "responses": ["Hasta luego!", "Hablamos luego!"]
             }
]}

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

### Herramientas de preprocesamiento de datos
Entre las tareas de procesamiento de texto en español se implementa:
- Quitar números
- Quitar símbolos de puntuación
- Quitar caracteres acentuados

In [ ]:
import re
import string

# El preprocesamento en castellano requiere más trabajo

def preprocess_clean_text(text):
    # pasar a minúsculas
    text = text.lower()
    # quitar números
    pattern = r'[0-9\n]'
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in (string.punctuation+"¡"+"¿")])
    # quitar caracteres con acento
    text = re.sub(r'[àáâä]', "a", text)
    text = re.sub(r'[éèêë]', "e", text)
    text = re.sub(r'[íìîï]', "i", text)
    text = re.sub(r'[òóôö]', "o", text)
    text = re.sub(r'[úùûü]', "u", text)
    return text

In [ ]:
string.punctuation + "¡" + "¿"

In [ ]:
preprocess_clean_text("¿cómo5!")

In [ ]:
words = []
classes = []
doc_X = []
doc_y = []
# Tokenizar cada "pattern" y agregar cada palabra al vocabulario (vocabulary)
# Los tokens que se toman de cada pattern se agrega a doc_X
# Cada tag se agrega a doc_y
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # trasformar el patron a tokens
        tokens = preprocess_clean_text(pattern).split(" ")
        # lematizar los tokens
        lemma_words = []
        for token in tokens:
            lemma = lemma_lookupTable.get(token)
            if lemma is not None:
                lemma_words.append(lemma)
            else:
                print("UNK:", token)
        
        if not lemma_words:
            continue
        
        words += lemma_words
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # Agregar el tag a las clases
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Elminar duplicados con "set" y ordenar el vocubulario y las clases por orden alfabético
vocab = sorted(set(words))
classes = sorted(set(classes))
len(vocab)

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
print("vocab:", vocab)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

In [ ]:
doc_y_encoded = [classes.index(label) for label in doc_y]
doc_y_encoded

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [ ]:
X_train = np.array(doc_X).reshape(-1, 1)
X_train

In [ ]:
X_train.shape

In [ ]:

y_train = tf.keras.utils.to_categorical(doc_y_encoded)
y_train[:4]

In [ ]:
output_shape = y_train.shape[1]
output_shape

In [ ]:
class CustomTextVectorization(tf.keras.layers.Layer):
    def __init__(self, vocab_data, lookupTable):
        super().__init__()
        keys, values = list(zip(*lookupTable.items()))
        table_init = tf.lookup.KeyValueTensorInitializer(keys, values)
        self.table = tf.lookup.StaticHashTable(table_init, "UNK")
        self.vectorize_layer = tf.keras.layers.TextVectorization(
            output_mode = "binary",
            split = self.custom_split,
            standardize=self.custom_lemmatization,
            max_tokens=len(vocab_data)+1, # vocab + UNK
            vocabulary=vocab_data
        )
        self.punctuation = string.punctuation + "¡" + "¿"

    def custom_lemmatization(self, input_string):
        # pasar a minúsculas
        output_string = tf.strings.lower(input_string, encoding='utf-8')
        # quitar números
        pattern = r'[0-9\n]' 
        output_string = tf.strings.regex_replace(output_string, pattern, '')
        # quitar signos de puntuacion
        output_string = tf.strings.regex_replace(
            output_string, "[%s]" % re.escape(self.punctuation), '')
        # quitar caracteres con acento
        output_string = tf.strings.regex_replace(output_string, r'[àáâä]', 'a')
        output_string = tf.strings.regex_replace(output_string, r'[éèêë]', 'e')
        output_string = tf.strings.regex_replace(output_string, r'[íìîï]', 'i')
        output_string = tf.strings.regex_replace(output_string, r'[òóôö]', 'o')
        output_string = tf.strings.regex_replace(output_string, r'[úùûü]', 'u')
        return output_string

    def custom_split(self, input_string):
        # split por espacios
        strings = tf.strings.split(input_string, sep=" ")
        # a cada token se lo lematiza
        strings = self.table.lookup(strings)
        return strings

    def call(self, inputs):
        return self.vectorize_layer(inputs)

custom_textVectorization = CustomTextVectorization(vocab, lemma_lookupTable)
custom_textVectorization.trainable = False
custom_textVectorization(["hola cómo!"])

In [ ]:
model_preprocess = Sequential()
model_preprocess.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model_preprocess.add(custom_textVectorization)
model_preprocess.build()
model_preprocess.summary()
model_preprocess.predict(X_train)

In [ ]:
# Entrenamiento del modelo DNN
# - Modelo secuencial
# - Con regularización
# - softmax y optimizador Adam
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(custom_textVectorization)
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation = "softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

In [ ]:
hist = model.fit(x=X_train, y=y_train, epochs=200, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['accuracy'], label='train')
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
responses = [[""]] * len(classes)
max_possible_responses = 0
for intent in data["intents"]:
    max_possible_responses = max(max_possible_responses, len(intent["responses"]))
    responses[classes.index(intent["tag"])] = intent["responses"]

# Para poder trabajar con las respuestas es necesario pasarlo a formato
# de matriz. Para eso, cada fila (tag) debe tener la misma cantidad de posibles
# respuestas
# Con el código a continuación se ajustan las respuestas y se repiten aquellas
# en donde la cantidad es menor a la requerida para formar una matriz
for i in range(len(responses)):
    if len(responses[i]) < max_possible_responses:
        responses[i] = list(np.resize(responses[i], max_possible_responses))

responses

In [ ]:
class CustomOutput(tf.keras.layers.Layer):
    def __init__(self, responses, threshold=0.2):
        super().__init__()
        self.responses = tf.constant(responses)
        self.maxval = self.responses.shape[1]
        self.default_output = tf.constant("Perdon, no pude entenderte")
        self.threshold =tf.constant(threshold)

    def call(self, inputs):
        score = tf.math.reduce_max(inputs)  # equivale a max()
        index = tf.math.argmax(inputs, axis=1)[0]  # equivale a argmax
        responses = tf.gather(self.responses, index)  # equivale a self.responses[index]
        
        # equivale a responses[random.randrange(0, maxval)]
        label = responses[tf.random.uniform(shape=(), minval=0, maxval=self.maxval, dtype=tf.int32)]

        # equivale a--> return label if score > 0.4 else "Perdón, no pude entenderte"
        output = tf.cond(score > self.threshold, lambda: label, lambda: self.default_output)

        # equivalente a reshape(1, -1) para retornar un vector (un array con texto dentro)
        output = tf.expand_dims(output, 0)
        return output

postprocess = CustomOutput(responses, 0.4)
postprocess(model.predict(["hola!"]))

In [ ]:
complete_model = Sequential()
complete_model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
complete_model.add(model)
complete_model.add(postprocess)
complete_model.build()
complete_model.summary()

In [ ]:
complete_model.predict(["Hola gente"])

In [ ]:
import os
MODEL_DIR = "chatbot"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))

tf.keras.models.save_model(
    complete_model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

In [ ]:
!zip -r chatbot.zip chatbot